# Information Retrieval: BM25 vs Jina Embeddings v4

This notebook compares two retrieval approaches:
- **BM25**: Traditional lexical (keyword-based) retrieval
- **Jina Embeddings v4**: Modern semantic (meaning-based) retrieval

We'll explore their strengths and weaknesses with carefully designed examples.

## Setup

In [30]:


%%capture
%pip install pandas==2.3.3

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [31]:
import sys
from pathlib import Path

# Add parent directory to path so we can import from retrievers
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd
from retrievers.embeddings import JinaEmbedder, DummyEmbedder, EmbeddingRetriever
from retrievers.bm25 import BM25
from metrics import precision_at_k, mrr, ndcg_at_k
import warnings
warnings.filterwarnings('ignore', message='.*urllib3 v2.*')

## Test Corpus

We've designed this corpus to highlight different retrieval scenarios:
1. **Exact keyword matches** - BM25 should excel
2. **Semantic/paraphrased queries** - Embeddings should excel
3. **Technical terms** - BM25's strength with rare terms
4. **Conceptual understanding** - Embeddings' strength

In [32]:
corpus = [
    {
        "doc_id": "climate1",
        "text": "Climate change is causing rising sea levels and extreme weather events, "
               "threatening coastal cities worldwide. Scientists warn of increasing hurricanes and floods."
    },
    {
        "doc_id": "climate2", 
        "text": "Renewable energy sources like solar and wind power are essential for "
               "reducing greenhouse gas emissions and combating global warming."
    },
    {
        "doc_id": "ml1",
        "text": "Machine learning algorithms can identify patterns in large datasets and "
               "make predictions based on training data using neural networks."
    },
    {
        "doc_id": "ml2",
        "text": "Deep neural networks use multiple layers to learn hierarchical "
               "representations of data for complex tasks like image recognition."
    },
    {
        "doc_id": "ml3",
        "text": "Artificial intelligence systems can now understand natural language, "
               "recognize objects in images, and even generate creative content."
    },
    {
        "doc_id": "space1",
        "text": "The James Webb Space Telescope is revealing unprecedented details about "
               "distant galaxies and the early universe using infrared technology."
    },
    {
        "doc_id": "quantum1",
        "text": "Quantum computing leverages quantum entanglement and superposition to solve "
               "complex computational problems exponentially faster than classical computers."
    },
    {
        "doc_id": "bio1",
        "text": "CRISPR gene editing technology enables precise modifications to DNA sequences, "
               "opening new possibilities for treating genetic diseases and improving crops."
    }
]

# Display corpus
df_corpus = pd.DataFrame(corpus)
print(f"📚 Corpus: {len(corpus)} documents\n")
df_corpus

📚 Corpus: 8 documents



,doc_id,text
0,climate1,Climate change is causing rising sea levels an...
1,climate2,Renewable energy sources like solar and wind p...
2,ml1,Machine learning algorithms can identify patte...
3,ml2,Deep neural networks use multiple layers to le...
4,ml3,Artificial intelligence systems can now unders...
5,space1,The James Webb Space Telescope is revealing un...
6,quantum1,Quantum computing leverages quantum entangleme...
7,bio1,CRISPR gene editing technology enables precise...


## Test Queries

Each query is designed to test specific retrieval characteristics:

### BM25 Strengths:
- **Exact keywords** (queries 1, 4, 5)
- **Rare technical terms**

### Embedding Strengths:
- **Semantic similarity** (queries 2, 7)
- **Paraphrasing** (query 6)
- **Conceptual understanding** (query 3)

In [33]:
queries = [
    {
        "id": "q1",
        "query": "quantum entanglement superposition",
        "expected": "quantum1",
        "type": "Exact keywords (BM25 strength)",
        "description": "Contains rare, technical terms that appear in only one document"
    },
    {
        "id": "q2",
        "query": "global warming and rising temperatures",
        "expected": "climate2",
        "type": "Semantic similarity (Embedding strength)",
        "description": "'global warming' is semantically similar to 'greenhouse gas emissions'"
    },
    {
        "id": "q3",
        "query": "How do computers learn from data?",
        "expected": "ml1",
        "type": "Conceptual question (Embedding strength)",
        "description": "Question form, no exact keywords but conceptually about machine learning"
    },
    {
        "id": "q4",
        "query": "CRISPR DNA editing",
        "expected": "bio1",
        "type": "Exact technical terms (BM25 strength)",
        "description": "Specific acronym and technical terms"
    },
    {
        "id": "q5",
        "query": "James Webb infrared telescope",
        "expected": "space1",
        "type": "Multi-keyword match (BM25 strength)",
        "description": "All three keywords appear in target document"
    },
    {
        "id": "q6",
        "query": "AI that understands human language",
        "expected": "ml3",
        "type": "Paraphrasing (Embedding strength)",
        "description": "Different words, same meaning as 'understand natural language'"
    },
    {
        "id": "q7",
        "query": "environmental impact of CO2",
        "expected": "climate2",
        "type": "Conceptual understanding (Embedding strength)",
        "description": "CO2 → greenhouse gas, conceptual link without exact words"
    },
]

df_queries = pd.DataFrame(queries)
print(f"🔍 Queries: {len(queries)} test cases\n")
df_queries[["id", "query", "type", "expected"]]

🔍 Queries: 7 test cases



,id,query,type,expected
0,q1,quantum entanglement superposition,Exact keywords (BM25 strength),quantum1
1,q2,global warming and rising temperatures,Semantic similarity (Embedding strength),climate2
2,q3,How do computers learn from data?,Conceptual question (Embedding strength),ml1
3,q4,CRISPR DNA editing,Exact technical terms (BM25 strength),bio1
4,q5,James Webb infrared telescope,Multi-keyword match (BM25 strength),space1
5,q6,AI that understands human language,Paraphrasing (Embedding strength),ml3
6,q7,environmental impact of CO2,Conceptual understanding (Embedding strength),climate2


## Initialize Retrievers

### 1. BM25 Retriever

In [34]:
print("📊 Initializing BM25 retriever...")
bm25 = BM25()
bm25.fit(corpus)
print("✓ BM25 ready!")

📊 Initializing BM25 retriever...
✓ BM25 ready!


### 2. Jina Embeddings Retriever

**Note:** First run will download ~7.5GB model. Subsequent runs load from cache.

In [35]:
jina_retriever = None
jina_embedder = None

try:
    print("🚀 Loading Jina embeddings v4 model...")
    print("   (This may take a few minutes on first run)\n")
    
    jina_embedder = JinaEmbedder(model_name="jinaai/jina-embeddings-v4", task="retrieval")
    print("✓ Model loaded!")
    
    # Test encoding
    test_emb = jina_embedder.encode(["test"], prompt_name="passage")
    print(f"✓ Embedding dimension: {test_emb.shape[1]}")
    print(f"✓ Normalized: {np.allclose(np.linalg.norm(test_emb[0]), 1.0, rtol=1e-3)}")
    
    jina_retriever = EmbeddingRetriever(embedder=jina_embedder)
    jina_retriever.fit(corpus)
    print("✓ Jina retriever ready!\n")
    
except Exception as e:
    print(f"❌ Error loading Jina model: {e}")
    print("   Install dependencies: pip install -r requirements.txt")
    print("   Continuing with BM25 only...\n")

🚀 Loading Jina embeddings v4 model...
   (This may take a few minutes on first run)



Fetching 2 files: 100%|██████████| 2/2 [00:00<00:00, 25040.62it/s]


✓ Model loaded!
✓ Embedding dimension: 2048
✓ Normalized: True
✓ Jina retriever ready!



### 3. Dummy Embeddings (Baseline)

Random normalized embeddings for comparison.

In [36]:
print("🎲 Initializing Dummy embedder (baseline)...")
dummy_embedder = DummyEmbedder()
dummy_retriever = EmbeddingRetriever(embedder=dummy_embedder)
dummy_retriever.fit(corpus)
print("✓ Dummy retriever ready!")

🎲 Initializing Dummy embedder (baseline)...
✓ Dummy retriever ready!


## Run Retrieval Experiments

In [37]:
def run_retrieval(retriever, query, k=3, name="Retriever"):
    """Run retrieval and return results."""
    if retriever is None:
        return None
    
    try:
        results = retriever.rank(query, k=k)
        return results
    except Exception as e:
        print(f"Error with {name}: {e}")
        return None


def format_results(results, expected_doc=None):
    """Format results as dataframe with highlighting."""
    if results is None:
        return None
    
    df = pd.DataFrame(results, columns=["doc_id", "score"])
    df["rank"] = range(1, len(df) + 1)
    df["correct"] = df["doc_id"] == expected_doc if expected_doc else False
    df = df[["rank", "doc_id", "score", "correct"]]
    return df

## Comparison Results

For each query, we'll show:
- Top 3 results from each retriever
- Whether the expected document was retrieved
- Analysis of why each method succeeded or failed

In [38]:
# Store all results
all_results = []

for q in queries:
    print("=" * 80)
    print(f"Query {q['id']}: {q['query']}")
    print(f"Type: {q['type']}")
    print(f"Expected: {q['expected']} - {q['description']}")
    print("=" * 80)
    
    # BM25
    print("\n📊 BM25 Results:")
    bm25_results = run_retrieval(bm25, q['query'], k=3, name="BM25")
    bm25_df = format_results(bm25_results, q['expected'])
    if bm25_df is not None:
        display(bm25_df)
        bm25_correct = bm25_df[bm25_df['correct']].shape[0] > 0
        bm25_rank = bm25_df[bm25_df['correct']]['rank'].values[0] if bm25_correct else None
    else:
        bm25_correct = False
        bm25_rank = None
    
    # Jina Embeddings
    print("\n🚀 Jina Embeddings Results:")
    jina_results = run_retrieval(jina_retriever, q['query'], k=3, name="Jina")
    jina_df = format_results(jina_results, q['expected'])
    if jina_df is not None:
        display(jina_df)
        jina_correct = jina_df[jina_df['correct']].shape[0] > 0
        jina_rank = jina_df[jina_df['correct']]['rank'].values[0] if jina_correct else None
    else:
        print("   (Not available)")
        jina_correct = False
        jina_rank = None
    
    # Dummy (baseline)
    print("\n🎲 Dummy Embeddings Results:")
    dummy_results = run_retrieval(dummy_retriever, q['query'], k=3, name="Dummy")
    dummy_df = format_results(dummy_results, q['expected'])
    if dummy_df is not None:
        display(dummy_df)
        dummy_correct = dummy_df[dummy_df['correct']].shape[0] > 0
        dummy_rank = dummy_df[dummy_df['correct']]['rank'].values[0] if dummy_correct else None
    else:
        dummy_correct = False
        dummy_rank = None
    
    # Store results
    all_results.append({
        'query_id': q['id'],
        'query': q['query'],
        'type': q['type'],
        'expected': q['expected'],
        'bm25_correct': bm25_correct,
        'bm25_rank': bm25_rank,
        'jina_correct': jina_correct,
        'jina_rank': jina_rank,
        'dummy_correct': dummy_correct,
        'dummy_rank': dummy_rank,
    })
    
    print("\n\n")

Query q1: quantum entanglement superposition
Type: Exact keywords (BM25 strength)
Expected: quantum1 - Contains rare, technical terms that appear in only one document

📊 BM25 Results:


,rank,doc_id,score,correct
0,1,quantum1,5.626808,True
1,2,climate1,0.000000,False
2,3,climate2,0.000000,False



🚀 Jina Embeddings Results:


,rank,doc_id,score,correct
0,1,quantum1,0.654099,True
1,2,space1,0.435976,False
2,3,ml3,0.430316,False



🎲 Dummy Embeddings Results:


...

,rank,doc_id,score,correct
0,1,ml2,1.308225,False
1,2,climate1,1.192117,False
2,3,climate2,0.000000,True



🚀 Jina Embeddings Results:


,rank,doc_id,score,correct
0,1,climate1,0.650154,False
1,2,climate2,0.610414,True
2,3,space1,0.433448,False



🎲 Dummy Embeddings Results:


,rank,doc_id,score,correct
0,1,climate1,0.055336,False
1,2,bio1,0.031194,False
2,3,ml3,0.004299,False


## Summary Analysis

In [39]:
df_results = pd.DataFrame(all_results)

print("=" * 80)
print("SUMMARY: Retrieval Performance")
print("=" * 80)

# Overall accuracy
print("\n📊 Accuracy (found expected document in top 3):")
print(f"  BM25:            {df_results['bm25_correct'].sum()}/{len(queries)} = {df_results['bm25_correct'].mean():.1%}")
if jina_retriever:
    print(f"  Jina Embeddings: {df_results['jina_correct'].sum()}/{len(queries)} = {df_results['jina_correct'].mean():.1%}")
print(f"  Dummy (baseline): {df_results['dummy_correct'].sum()}/{len(queries)} = {df_results['dummy_correct'].mean():.1%}")

# By query type
print("\n📈 Performance by Query Type:")
display(df_results.groupby('type').agg({
    'bm25_correct': 'mean',
    'jina_correct': 'mean',
    'dummy_correct': 'mean'
}).round(2))

print("\n💡 Full Results Table:")
print("(Rank shows position 1-3 if found, '>3' if not in top 3)\n")

# Replace NaN with ">3" for better readability
df_results_display = df_results.copy()
rank_columns = ['bm25_rank', 'jina_rank', 'dummy_rank']
for col in rank_columns:
    if col in df_results_display.columns:
        df_results_display[col] = df_results_display[col].fillna('>3').astype(str).str.replace('.0', '', regex=False)

display(df_results_display)

SUMMARY: Retrieval Performance

📊 Accuracy (found expected document in top 3):
  BM25:            5/7 = 71.4%
  Jina Embeddings: 7/7 = 100.0%
  Dummy (baseline): 3/7 = 42.9%

📈 Performance by Query Type:


,bm25_correct,jina_correct,dummy_correct
type,,,
Conceptual question (Embedding strength),0.0,1.0,0.0
Conceptual understanding (Embedding strength),1.0,1.0,0.0
Exact keywords (BM25 strength),1.0,1.0,1.0
Exact technical terms (BM25 strength),1.0,1.0,1.0
Multi-keyword match (BM25 strength),1.0,1.0,0.0
Paraphrasing (Embedding strength),0.0,1.0,1.0
Semantic similarity (Embedding strength),1.0,1.0,0.0



💡 Full Results Table:
(Rank shows position 1-3 if found, '>3' if not in top 3)



,query_id,query,type,expected,bm25_correct,bm25_rank,jina_correct,jina_rank,dummy_correct,dummy_rank
0,q1,quantum entanglement superposition,Exact keywords (BM25 strength),quantum1,True,1,True,1,True,3
1,q2,global warming and rising temperatures,Semantic similarity (Embedding strength),climate2,True,1,True,2,False,>3
2,q3,How do computers learn from data?,Conceptual question (Embedding strength),ml1,False,>3,True,1,False,>3
3,q4,CRISPR DNA editing,Exact technical terms (BM25 strength),bio1,True,1,True,1,True,1
4,q5,James Webb infrared telescope,Multi-keyword match (BM25 strength),space1,True,1,True,1,False,>3
5,q6,AI that understands human language,Paraphrasing (Embedding strength),ml3,False,>3,True,1,True,3
6,q7,environmental impact of CO2,Conceptual understanding (Embedding strength),climate2,True,3,True,2,False,>3


## Detailed Metrics Analysis

Let's calculate standard IR metrics: **Precision@k**, **MRR** (Mean Reciprocal Rank), and **NDCG@k** (Normalized Discounted Cumulative Gain).


In [40]:
# Build qrels (query relevance judgments) from our queries
# Each query has exactly one expected document with relevance score 1
qrels = {}
for q in queries:
    qrels[q['id']] = {q['expected']: 1}

# Build run dictionaries (query_id -> ranked list of doc_ids) for each retriever
def build_run_dict(all_results):
    """Convert results to format expected by metrics: {qid: [ranked_doc_ids]}"""
    run = {}
    for result in all_results:
        qid = result['query_id']
        query_text = result['query']
        
        # Get top-3 results from each retriever
        bm25_results = bm25.rank(query_text, k=3)
        run[qid] = [doc_id for doc_id, score in bm25_results]
    return run

# Build runs for each retriever
print("📊 Calculating IR Metrics (P@3, MRR, NDCG@3)...\n")

# BM25 run
bm25_run = {}
for q in queries:
    results = bm25.rank(q['query'], k=3)
    bm25_run[q['id']] = [doc_id for doc_id, score in results]

# Jina run
jina_run = {}
if jina_retriever:
    for q in queries:
        results = jina_retriever.rank(q['query'], k=3)
        jina_run[q['id']] = [doc_id for doc_id, score in results]

# Dummy run
dummy_run = {}
for q in queries:
    results = dummy_retriever.rank(q['query'], k=3)
    dummy_run[q['id']] = [doc_id for doc_id, score in results]

# Calculate metrics for each retriever
def calculate_metrics(run, qrels, k=3):
    """Calculate average metrics across all queries."""
    p_sum = mrr_sum = ndcg_sum = 0.0
    n_queries = 0
    
    for qid, ranked_ids in run.items():
        if qid not in qrels:
            continue
        n_queries += 1
        p_sum += precision_at_k(ranked_ids, qrels[qid], k)
        mrr_sum += mrr(ranked_ids, qrels[qid])
        ndcg_sum += ndcg_at_k(ranked_ids, qrels[qid], k, method="exponential")
    
    if n_queries == 0:
        return {"P@k": 0.0, "MRR": 0.0, "NDCG@k": 0.0}
    
    return {
        f"P@{k}": round(p_sum / n_queries, 3),
        "MRR": round(mrr_sum / n_queries, 3),
        f"NDCG@{k}": round(ndcg_sum / n_queries, 3)
    }

# Calculate metrics for each retriever
k = 3
bm25_metrics = calculate_metrics(bm25_run, qrels, k=k)
jina_metrics = calculate_metrics(jina_run, qrels, k=k) if jina_retriever else {"P@3": 0.0, "MRR": 0.0, "NDCG@3": 0.0}
dummy_metrics = calculate_metrics(dummy_run, qrels, k=k)

# Create comparison table
metrics_comparison = pd.DataFrame({
    'BM25': bm25_metrics,
    'Jina Embeddings': jina_metrics,
    'Dummy (baseline)': dummy_metrics
}).T

print("=" * 80)
print("IR METRICS COMPARISON")
print("=" * 80)
print()
display(metrics_comparison)

print("\n📖 Metric Definitions:")
print(f"  • P@{k} (Precision@{k}): Fraction of relevant docs in top {k} results")
print("  • MRR (Mean Reciprocal Rank): Average of 1/rank of first relevant doc")
print(f"  • NDCG@{k}: Normalized DCG - measures ranking quality (0-1, higher=better)")
print("\n💡 All metrics averaged across all queries. Higher is better!")


📊 Calculating IR Metrics (P@3, MRR, NDCG@3)...



IR METRICS COMPARISON



,P@3,MRR,NDCG@3
BM25,0.238,0.619,0.643
Jina Embeddings,0.333,0.857,0.895
Dummy (baseline),0.143,0.238,0.286



📖 Metric Definitions:
  • P@3 (Precision@3): Fraction of relevant docs in top 3 results
  • MRR (Mean Reciprocal Rank): Average of 1/rank of first relevant doc
  • NDCG@3: Normalized DCG - measures ranking quality (0-1, higher=better)

💡 All metrics averaged across all queries. Higher is better!


### Per-Query Metrics Breakdown

Let's examine metrics for each individual query to understand where each retriever excels.


In [41]:
# Calculate per-query metrics
per_query_metrics = []

for q in queries:
    qid = q['id']
    qtype = q['type']
    
    # BM25 metrics
    bm25_ranked = bm25_run[qid]
    bm25_p = precision_at_k(bm25_ranked, qrels[qid], k=1)
    bm25_mrr = mrr(bm25_ranked, qrels[qid])
    bm25_ndcg = ndcg_at_k(bm25_ranked, qrels[qid], k=3, method="exponential")
    
    # Jina metrics
    if jina_retriever:
        jina_ranked = jina_run[qid]
        jina_p = precision_at_k(jina_ranked, qrels[qid], k=1)
        jina_mrr = mrr(jina_ranked, qrels[qid])
        jina_ndcg = ndcg_at_k(jina_ranked, qrels[qid], k=3, method="exponential")
    else:
        jina_p = jina_mrr = jina_ndcg = 0.0
    
    # Dummy metrics
    dummy_ranked = dummy_run[qid]
    dummy_p = precision_at_k(dummy_ranked, qrels[qid], k=1)
    dummy_mrr = mrr(dummy_ranked, qrels[qid])
    dummy_ndcg = ndcg_at_k(dummy_ranked, qrels[qid], k=3, method="exponential")
    
    per_query_metrics.append({
        'Query ID': qid,
        'Query Type': qtype,
        'BM25 P@1': round(bm25_p, 2),
        'BM25 MRR': round(bm25_mrr, 2),
        'BM25 NDCG@3': round(bm25_ndcg, 2),
        'Jina P@1': round(jina_p, 2),
        'Jina MRR': round(jina_mrr, 2),
        'Jina NDCG@3': round(jina_ndcg, 2),
        'Dummy P@1': round(dummy_p, 2),
        'Dummy MRR': round(dummy_mrr, 2),
        'Dummy NDCG@3': round(dummy_ndcg, 2),
    })

df_per_query = pd.DataFrame(per_query_metrics)

# Calculate aggregate scores (mean across all queries)
aggregate_row = {
    'Query ID': 'AVERAGE',
    'Query Type': 'All queries',
    'BM25 P@1': round(df_per_query['BM25 P@1'].mean(), 2),
    'BM25 MRR': round(df_per_query['BM25 MRR'].mean(), 2),
    'BM25 NDCG@3': round(df_per_query['BM25 NDCG@3'].mean(), 2),
    'Jina P@1': round(df_per_query['Jina P@1'].mean(), 2),
    'Jina MRR': round(df_per_query['Jina MRR'].mean(), 2),
    'Jina NDCG@3': round(df_per_query['Jina NDCG@3'].mean(), 2),
    'Dummy P@1': round(df_per_query['Dummy P@1'].mean(), 2),
    'Dummy MRR': round(df_per_query['Dummy MRR'].mean(), 2),
    'Dummy NDCG@3': round(df_per_query['Dummy NDCG@3'].mean(), 2),
}

# Add aggregate row to the dataframe
df_per_query_with_avg = pd.concat([df_per_query, pd.DataFrame([aggregate_row])], ignore_index=True)

# Display with better formatting
print("📊 Per-Query Metrics Breakdown:\n")
display(df_per_query_with_avg)

# Highlight best performing retriever for each query type
print("\n🏆 Best Performer by Query Type (based on NDCG@3):")
for qtype in df_per_query['Query Type'].unique():
    type_rows = df_per_query[df_per_query['Query Type'] == qtype]
    avg_bm25 = type_rows['BM25 NDCG@3'].mean()
    avg_jina = type_rows['Jina NDCG@3'].mean()
    avg_dummy = type_rows['Dummy NDCG@3'].mean()
    
    best = max([('BM25', avg_bm25), ('Jina', avg_jina), ('Dummy', avg_dummy)], key=lambda x: x[1])
    print(f"  • {qtype[:50]}... → {best[0]} ({best[1]:.2f})")


📊 Per-Query Metrics Breakdown:



,Query ID,Query Type,BM25 P@1,BM25 MRR,BM25 NDCG@3,Jina P@1,Jina MRR,Jina NDCG@3,Dummy P@1,Dummy MRR,Dummy NDCG@3
0,q1,Exact keywords (BM25 strength),1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.33,0.50
1,q2,Semantic similarity (Embedding strength),1.00,1.00,1.00,0.00,0.50,0.63,0.00,0.00,0.00
2,q3,Conceptual question (Embedding strength),0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.00,0.00
3,q4,Exact technical terms (BM25 strength),1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
4,q5,Multi-keyword match (BM25 strength),1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.00
5,q6,Paraphrasing (Embedding strength),0.00,0.00,0.00,1.00,1.00,1.00,0.00,0.33,0.50
6,q7,Conceptual understanding (Embedding strength),0.00,0.33,0.50,0.00,0.50,0.63,0.00,0.00,0.00
7,AVERAGE,All queries,0.57,0.62,0.64,0.71,0.86,0.89,0.14,0.24,0.29



🏆 Best Performer by Query Type (based on NDCG@3):
  • Exact keywords (BM25 strength)... → BM25 (1.00)
  • Semantic similarity (Embedding strength)... → BM25 (1.00)
  • Conceptual question (Embedding strength)... → Jina (1.00)
  • Exact technical terms (BM25 strength)... → BM25 (1.00)
  • Multi-keyword match (BM25 strength)... → BM25 (1.00)
  • Paraphrasing (Embedding strength)... → Jina (1.00)
  • Conceptual understanding (Embedding strength)... → Jina (0.63)


GRADED RELEVANCE EXAMPLE: Query 'global warming and rising temperatures'

📋 Relevance Judgments:
  • climate2 (rel=3): Highly relevant - about greenhouse gas emissions
  • climate1 (rel=2): Relevant - about climate change impacts
  • space1   (rel=1): Somewhat relevant - environmental context

🏆 Actual Rankings:

  BM25: ['climate2', 'climate1', 'ml3']
  Jina: ['climate1', 'climate2', 'space1']

📊 NDCG@3 with Graded Relevance:
  BM25: 0.947
  Jina: 0.843

💡 Why Graded Relevance Matters:

  Example: Perfect vs. Good ranking
  ────────────────────────────────
  Ranking A: [climate2(3), climate1(2), space1(1)]  ← Perfect order
  Ranking B: [climate1(2), climate2(3), space1(1)]  ← Swapped top 2

  NDCG@3 (Perfect): 1.000
  NDCG@3 (Swapped): 0.843
  Difference: 0.157

  ➜ NDCG penalizes systems that rank highly relevant docs lower!

  With Binary Relevance (both climate docs = 1):
  NDCG@3 (Perfect): 1.000
  NDCG@3 (Swapped): 1.000
  Difference: 0.000

  ➜ Binary relevance can't distinguish

In [43]:
# Load graded qrels from our data files
import json
from pathlib import Path

# Load qrels with graded relevance (1-3)
qrels_path = Path.cwd().parent / "data" / "qrels.jsonl"
graded_qrels_full = {}

try:
    with open(qrels_path, 'r') as f:
        for line in f:
            item = json.loads(line.strip())
            qid = item['qid']
            doc_id = item['doc_id']
            rel = item['rel']
            
            if qid not in graded_qrels_full:
                graded_qrels_full[qid] = {}
            graded_qrels_full[qid][doc_id] = rel
    
    print("✓ Loaded graded qrels from data/qrels.jsonl")
    print(f"  {len(graded_qrels_full)} queries with graded relevance judgments\n")
    
    # Show example of graded qrels
    print("📋 Example Graded Relevance Judgments:")
    for qid in list(graded_qrels_full.keys())[:3]:
        print(f"\n  {qid}:")
        for doc_id, rel in sorted(graded_qrels_full[qid].items(), key=lambda x: x[1], reverse=True):
            print(f"    {doc_id}: {rel}")
    
except FileNotFoundError:
    print("⚠️  Could not find data/qrels.jsonl - using example data only")
    graded_qrels_full = graded_qrels_example

print("\n" + "=" * 80)
print("COMPARISON: Binary vs. Graded Relevance Evaluation")
print("=" * 80)

# Build binary qrels (convert all relevance > 0 to 1)
binary_qrels_full = {}
for qid, docs in graded_qrels_full.items():
    binary_qrels_full[qid] = {doc_id: 1 for doc_id, rel in docs.items() if rel > 0}

# Calculate metrics with both binary and graded relevance
def evaluate_with_relevance_type(run, qrels, k=3, rel_type="binary"):
    """Calculate metrics, showing which relevance type is used."""
    ndcg_sum = 0.0
    n_queries = 0
    
    for qid, ranked_ids in run.items():
        if qid not in qrels:
            continue
        n_queries += 1
        ndcg_sum += ndcg_at_k(ranked_ids, qrels[qid], k, method="exponential")
    
    return {
        f"NDCG@{k} ({rel_type})": round(ndcg_sum / max(1, n_queries), 3)
    }

# Evaluate BM25 with both relevance types
bm25_metrics_binary = evaluate_with_relevance_type(bm25_run, binary_qrels_full, k=3, rel_type="binary")
bm25_metrics_graded = evaluate_with_relevance_type(bm25_run, graded_qrels_full, k=3, rel_type="graded")

# Evaluate Jina with both relevance types  
jina_metrics_binary = evaluate_with_relevance_type(jina_run, binary_qrels_full, k=3, rel_type="binary") if jina_run else {"NDCG@3 (binary)": 0.0}
jina_metrics_graded = evaluate_with_relevance_type(jina_run, graded_qrels_full, k=3, rel_type="graded") if jina_run else {"NDCG@3 (graded)": 0.0}

# Create comparison DataFrame
comparison_data = {
    'BM25': {
        'NDCG@3 (Binary)': bm25_metrics_binary['NDCG@3 (binary)'],
        'NDCG@3 (Graded)': bm25_metrics_graded['NDCG@3 (graded)'],
    },
    'Jina Embeddings': {
        'NDCG@3 (Binary)': jina_metrics_binary['NDCG@3 (binary)'],
        'NDCG@3 (Graded)': jina_metrics_graded['NDCG@3 (graded)'],
    }
}

df_comparison = pd.DataFrame(comparison_data).T
print()
display(df_comparison)

print("\n💡 Key Insights:")
print("  • Binary relevance: Treats all relevant docs equally (1 or 0)")
print("  • Graded relevance: Distinguishes between 'good' (1), 'better' (2), and 'best' (3)")
print("  • NDCG with graded relevance rewards ranking highly relevant docs first")
print("  • Graded NDCG typically < Binary NDCG because it's more strict about ranking order")
print("\n✨ Use graded relevance when:")
print("  • You have documents with varying quality/relevance")
print("  • Ranking order matters beyond just 'relevant vs not relevant'")
print("  • You want to incentivize systems to prioritize the best matches")


⚠️  Could not find data/qrels.jsonl - using example data only

COMPARISON: Binary vs. Graded Relevance Evaluation



,NDCG@3 (Binary),NDCG@3 (Graded)
BM25,0.617,0.633
Jina Embeddings,0.883,0.895



💡 Key Insights:
  • Binary relevance: Treats all relevant docs equally (1 or 0)
  • Graded relevance: Distinguishes between 'good' (1), 'better' (2), and 'best' (3)
  • NDCG with graded relevance rewards ranking highly relevant docs first
  • Graded NDCG typically < Binary NDCG because it's more strict about ranking order

✨ Use graded relevance when:
  • You have documents with varying quality/relevance
  • Ranking order matters beyond just 'relevant vs not relevant'
  • You want to incentivize systems to prioritize the best matches


## Key Insights

### BM25 Strengths:
✅ Excellent with **exact keyword matches**  
✅ Strong on **rare technical terms** (high IDF)  
✅ Fast and efficient  
✅ Interpretable (you can see which terms matched)  

### BM25 Weaknesses:
❌ No semantic understanding ("global warming" ≠ "greenhouse gas")  
❌ Struggles with **paraphrasing**  
❌ Can't handle **conceptual queries**  
❌ Vocabulary mismatch problems  

### Jina Embeddings Strengths:
✅ Understands **semantic similarity**  
✅ Handles **paraphrasing** well  
✅ Works with **conceptual queries**  
✅ Robust to vocabulary mismatch  
✅ Multilingual and multimodal (text + images)  

### Jina Embeddings Weaknesses:
❌ Computationally expensive  
❌ Requires GPU for fast inference at scale  
❌ Less interpretable (black box)  
❌ May miss exact matches if not in training data  

### Best Practice:
🎯 **Hybrid retrieval** - Combine both approaches!
- Use BM25 for keyword matches
- Use embeddings for semantic understanding
- Merge and re-rank results

## Similarity Inspection (Jina Embeddings)

Let's visualize how Jina understands semantic relationships.

In [44]:
if jina_embedder:
    print("🔬 Pairwise Semantic Similarities\n")
    
    pairs = [
        ("climate change", "global warming"),
        ("machine learning", "artificial intelligence"),
        ("neural networks", "deep learning"),
        ("quantum computing", "classical computing"),
        ("climate change", "quantum computing"),  # Unrelated
    ]
    
    similarity_data = []
    
    for text1, text2 in pairs:
        emb1 = jina_embedder.encode([text1], prompt_name="passage")[0]
        emb2 = jina_embedder.encode([text2], prompt_name="passage")[0]
        
        # Cosine similarity (dot product for normalized vectors)
        similarity = float(np.dot(emb1, emb2))
        
        similarity_data.append({
            'text1': text1,
            'text2': text2,
            'similarity': similarity,
            'interpretation': (
                'Very similar' if similarity > 0.8 else
                'Similar' if similarity > 0.6 else
                'Somewhat similar' if similarity > 0.4 else
                'Different'
            )
        })
    
    df_similarity = pd.DataFrame(similarity_data)
    display(df_similarity)
else:
    print("⚠️  Jina embeddings not available")

🔬 Pairwise Semantic Similarities



,text1,text2,similarity,interpretation
0,climate change,global warming,0.915079,Very similar
1,machine learning,artificial intelligence,0.837106,Very similar
2,neural networks,deep learning,0.827342,Very similar
3,quantum computing,classical computing,0.869445,Very similar
4,climate change,quantum computing,0.731588,Similar


## Conclusion

Both BM25 and neural embeddings have their place in modern IR systems:

- **BM25**: Fast, interpretable, great for exact matches
- **Embeddings**: Semantic understanding, handles paraphrasing
- **Best approach**: Hybrid systems that combine both strengths

For production systems, consider:
1. First-stage retrieval with BM25 (fast, broad recall)
2. Re-ranking with embeddings (precise, semantic)
3. Evaluation metrics (Precision@k, MRR, NDCG)